[参考実装1](https://qiita.com/makaishi2/items/aa88ccdc87af3e45edd7)

# 環境セットアップ

## 作業ディレクトリ作成

In [ ]:
# 作業ディレクトリ設定
%cd /content
!mkdir forecasting

## ライブラリのインストール

In [ ]:
!pip install pystan==2.19.1.1
!pip install prophet

## ライブラリのインポート

In [ ]:
import pandas as pd
pd.options.display.max_columns = 100 # 表示列上限変更

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

import datetime
import itertools
from tqdm import tqdm

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import cross_validation
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import performance_metrics

from sklearn.metrics import mean_squared_error, mean_absolute_error


# データのセットアップ

## ダウンロード
[厚生労働省-オープンデータ](https://www.mhlw.go.jp/stf/covid-19/open-data.html)

In [ ]:
%cd /content/forecasting
!mkdir datasets

!wget -c https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv \
      -O ./datasets/newly_confirmed_cases_daily.csv

In [ ]:
%cd /content/forecasting


daily_df = pd.read_csv(
    "./datasets/newly_confirmed_cases_daily.csv",
    parse_dates=["Date"])

# 最終行から5行目まで表示
daily_df.tail(5)

## 移動平均作成関数定義

In [ ]:
def moving_average(df, column, window_size):
  '''
  移動平均作成
  '''
  df[column + '_macd_' + str(window_size)] = \
    df[column].transform(lambda x: x.rolling(window=window_size).mean()).astype(np.float16)
  return df

# データの可視化

## 要約統計量

In [ ]:
daily_df.describe()

## 欠損値の確認

In [ ]:
# nullの個数を計算
num_of_null = daily_df.isnull().values.sum()
print('before:', num_of_null)


# 厚生労働省のcsvの更新タイミングによってはNanが入った行が存在するため
# Nanが入った行を削除
daily_df = daily_df.dropna(how='any')
num_of_null = daily_df.isnull().values.sum()
print('after:', num_of_null)

## 東京の陽性者数をプロット

In [ ]:
# 描画する列名
show_columns = ["Tokyo"]

fig, ax = plt.subplots(figsize=(16,10), dpi=150)
# グラフのタイトル設定
ax.set_title("Number of positive people")

for i, column in enumerate(show_columns):
  # x軸Date, y軸columnの棒グラフ作成
  ax.bar(
      daily_df["Date"], daily_df[column], 
      color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i], label=column)
  
# gridを描画
ax.grid(True)
# 凡例を描画
ax.legend(loc=0)

plt.show()


## 移動平均を追加して可視化

In [ ]:
# 描画する列名
show_columns = ["Tokyo"]
macd_windows = [7, 28, 54]

fig, ax = plt.subplots(figsize=(16,10), dpi=150)
# グラフのタイトル設定
ax.set_title("Number of positive people")

for i, column in enumerate(show_columns):
  # x軸Date, y軸columnの棒グラフ作成
  ax.bar(
      daily_df["Date"], daily_df[column], 
      color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i], label=column)
  for j, window in enumerate(macd_windows):
    ax.plot(
      daily_df["Date"], moving_average(daily_df, column, window)[column + '_macd_' + str(window)], 
      color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i+(j+1)], label=column + '_macd_' + str(window))
  
# gridを描画
ax.grid(True)
# 凡例を描画
ax.legend(loc=0)

plt.show()

## 成分分解
[参考資料](https://blog.amedama.jp/entry/sm-decompose-series)


In [ ]:
res = sm.tsa.seasonal_decompose(
    daily_df["Tokyo"],
    freq=12,
    model="additive")

# 上から原系列、トレンド、季節性、残差
fig = res.plot()

# トレーニング

## train, testデータに分割

In [ ]:
def build_dataset(df, ues_columns, date_column, target_column, test_rate=0.2, test_date=None):
  # 元データの意図しない変更を防ぐためdeep copy
  temp_df = df.copy(deep=True)

  #####################################
  # Prophetが求めるデータ構造に変換
  #####################################
  # date, valueの2列のみ抽出
  temp_df = temp_df[ues_columns]
  temp_df = temp_df.rename(columns={date_column:'ds', target_column: 'y'})

  #####################################
  # train, testに分割
  #####################################
  if test_date== None:
    test_len = int(len(temp_df)*test_rate)
    test_idx = len(temp_df) - test_len

    # test_idxを基準に分割
    train_df = temp_df.iloc[:test_idx, :]
    test_df = temp_df.iloc[test_idx:, :]
  else:
    train_df = temp_df[temp_df['ds'] < test_date]
    test_df = temp_df[test_date <= temp_df['ds']]

  return train_df, test_df

In [ ]:
len(daily_df)

In [ ]:
train_df, test_df = build_dataset(daily_df, ["Date", "Tokyo"], "Date", "Tokyo", 0.1)
display(train_df, test_df)

## fit

In [ ]:
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,

    # 増加傾向の場合はmultiplicative
    seasonality_mode='additive' # additive or multiplicative
    )

#fit the model with your dataframe
model.fit(train_df.loc[:,['ds', 'y']])

# テスト

In [ ]:
# test_dataのdsに合わせてpredict
test_fcst = model.predict(
    df = test_df
)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5), dpi=150)

fig = model.plot(
    test_fcst,
    ax=ax)

# グラフのタイトル設定
ax.set_title("Number of positive people predict.")
# x軸の上限下限設定
ax.set_xlim([test_fcst.head(1).ds, test_fcst.tail(1).ds])
ax.plot(test_df["ds"], test_df["y"], color='r', label="true")
# gridを描画
ax.grid(True)
# 凡例を描画
ax.legend(loc=0)

plt.show()

## エラー率確認

## eval
Mean Squared Error:MSE(平均二乗誤差)<br>
*   実際の値と予測値の絶対値の2乗の平均<br>
小さいほど良い<br>

Mean Absolute Error:MAE(平均絶対誤差)<br>
*   実際の値と予測値の絶対差の平均<br>
小さいほど良い<br>

Mean Absolute Percentage Error:MAPE(平均絶対パーセント誤差)<br>
*   実際の値と予測値との差を、実際の値で割った値(=パーセント誤差)」の絶対値を計算し、その総和をデータ数で割った値(=平均値)<br>
小さいほど良い<br>
  

参考:  
https://aizine.ai/rmse-rmsle1114/#toc3  
https://atmarkit.itmedia.co.jp/ait/articles/2106/09/news028.html


In [ ]:
def calc_error_rates(y_true, y_pred):
  mse = mean_squared_error(
      y_true = y_true,
      y_pred = y_pred)
  mae = mean_absolute_error(
      y_true = y_true,
      y_pred = y_pred)
  
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  return mse, mae, mape

In [ ]:
mse, mae, mape = calc_error_rates(
    y_true = test_df.y,
    y_pred = test_fcst.yhat
)
print("MSE:%f, MAE:%f, MAPE:%f" % (mse, mae, mape) )

# 精度改善

In [ ]:
improve_df = daily_df.loc[:, ["Date", "Tokyo"]]

In [ ]:
# ほとんど横ばいの2020年7月までを含めない
# improve_df = improve_df[datetime.datetime(2020, 7, 1) <= improve_df['Date'] ]

## 祝日・イベント追加


In [ ]:
# Prophetのadd_country_holidaysで祝日を追加することも可能だが
# 正確性を重視して内閣府掲載の祝日情報を取得
holidays_df = pd.read_csv(
    "https://www8.cao.go.jp/chosei/shukujitsu/syukujitsu.csv", 
    encoding="SHIFT_JIS",
    parse_dates=["国民の祝日・休日月日"])

# 陽性者数の計測が始まった時点からの祝日・休日情報のみ取得
holidays_df = holidays_df[improve_df["Date"].iloc[0] <= holidays_df["国民の祝日・休日月日"]]
holidays_df

In [ ]:
# prophetの求めるデータ構造に変換
holidays = pd.DataFrame({
    'holiday': 'jp_holidays',
    'ds': pd.to_datetime(holidays_df["国民の祝日・休日月日"]),
    'lower_window': 0,
    'upper_window': 0,
})

# トレーニング

## train, testデータに分割

In [ ]:
train_df, test_df = build_dataset(improve_df, ["Date", "Tokyo"], "Date", "Tokyo", test_rate=0.1)
display(train_df, test_df)

## fit

In [ ]:
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,

    # 増加傾向の場合はmultiplicative
    seasonality_mode='additive', # additive or multiplicative

    # cap, floorを使用する場合に設定
    # growth='logistic', # specify logistic growth
    # 国民の祝日・休日情報
    holidays = holidays,  
    )

# 四半期ごとの季節性追加 
# 引数は下記参照
# https://facebook.github.io/prophet/docs/multiplicative_seasonality.html
# model.add_seasonality(name = 'quarterly', period=91.25, fourier_order=8, mode='additive')

#fit the model with your dataframe
model.fit(train_df.loc[:,['ds', 'y']])

# テスト

In [ ]:
# test_dataのdsに合わせてpredict
test_fcst = model.predict(
    df = test_df
)

fig, ax = plt.subplots(figsize=(15,5), dpi=150)

fig = model.plot(
    test_fcst,
    ax=ax)

# グラフのタイトル設定
ax.set_title("Number of positive people predict.")
# x軸の上限下限設定
ax.set_xlim([test_fcst.head(1).ds, test_fcst.tail(1).ds])
ax.plot(test_df["ds"], test_df["y"], color='r', label="true")
# gridを描画
ax.grid(True)
# 凡例を描画
ax.legend(loc=0)

plt.show()

## エラー率確認

In [ ]:
mse, mae, mape = calc_error_rates(
    y_true = test_df.y,
    y_pred = test_fcst.yhat
)
print("MSE:%f, MAE:%f, MAPE:%f" % (mse, mae, mape) )

# ハイパーパラメータチューニング

In [ ]:
# MSE:5836639.659307, MAE:2070.112366, MAPE:46.448763
# 'changepoint_prior_scale': 0.005, 
# 'seasonality_prior_scale': 5.0, 
# 'holidays_prior_scale': 20.0, 
# 'seasonality_mode': 'additive', 
# 'changepoint_range': 0.8,

# MSE:5839819.916396, MAE:2079.609590, MAPE:47.021056
# 'changepoint_prior_scale': 0.005, 
# 'seasonality_prior_scale': 1.0, 
# 'holidays_prior_scale': 25.0, 
# 'seasonality_mode': 'additive', 
# 'changepoint_range': 0.7

# MSE:5902161.127703, MAE:2066.354548, MAPE:45.723594
# 'changepoint_prior_scale': 0.005, 
# 'seasonality_prior_scale': 5.0, 
# 'holidays_prior_scale': 25.0, 
# 'seasonality_mode': 'additive', 
# 'changepoint_range': 0.8


param_grid = {
    # Able to tune
    'changepoint_prior_scale':  [0.005], # default 0.05
    'seasonality_prior_scale':  [1.0, 2.5, 5.0], # default 10
    'holidays_prior_scale':     [20.0, 25.0, 30.0], # default 10
    'seasonality_mode' :        ['additive'], # additive or multiplicative
    # May be tune
    'changepoint_range':        [0.8], # default 0.8
    'holidays':           [holidays],
    # 下記は下手にいじるよりseasonality_prior_scaleで調整した方が良い
    'yearly_seasonality': [True], # default 10
    'weekly_seasonality': [True], # default 10
    'daily_seasonality':  [True], # default 10
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

rmses = []  # Store the RMSEs for each params here

In [ ]:
train_df, test_df = build_dataset(improve_df, ["Date", "Tokyo"], "Date", "Tokyo", test_date=datetime.datetime(2022, 3, 9))

for i in tqdm( range(len(all_params)) ):
# for i, params in enumerate(all_params):
    params = all_params[i]
    model = Prophet(**params)

    model.fit(train_df.loc[:,['ds', 'y']]) 

    # parallel processes, threads, dask
    # cross validationを400日目から開始、86日間をテストデータとし、検証後54日スライド
    df_cv = cross_validation(model, initial='400 days', period='54 days', horizon = '86 days', parallel="threads")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

In [ ]:
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
best_params = all_params[np.argmin(rmses)]
print(best_params)

## best paramでモデル生成

In [ ]:
model = Prophet(**best_params)

model.fit(train_df.loc[:,['ds', 'y']]) 

# テスト

In [ ]:
# test_dataのdsに合わせてpredict
test_fcst = model.predict(
    df = test_df
)

fig, ax = plt.subplots(figsize=(15,5), dpi=150)

fig = model.plot(
    test_fcst,
    ax=ax)

# グラフのタイトル設定
ax.set_title("Number of positive people predict.")
# x軸の上限下限設定
ax.set_xlim([test_fcst.head(1).ds, test_fcst.tail(1).ds])
ax.plot(test_df["ds"], test_df["y"], color='r', label="true")
# gridを描画
ax.grid(True)
# 凡例を描画
ax.legend(loc=0)

plt.show()

## エラー率確認

In [ ]:
mse, mae, mape = calc_error_rates(
    y_true = test_df.y,
    y_pred = test_fcst.yhat
)
print("MSE:%f, MAE:%f, MAPE:%f" % (mse, mae, mape) )